In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset

# 右端项f(x, y) = ax + by
def f(x, y, a, b):
    return a * x + b * y

# 求解区域
rangee = [0, 1]

def psi(f, a, b, xi, xi1, yi, hi):
    return (f(xi, yi, a, b) + f(xi1, yi + hi * f(xi, yi, a, b), a, b)) / 2

def heun_method(f, a, b, c, dx, rangee):
    N = int((rangee[1] - rangee[0]) / dx)
    x = np.zeros(N+1)
    y = np.zeros(N+1)
    x[0] = rangee[0]
    y[0] = c
    for i in range(N):
        x[i+1] = x[i] + dx
        y[i+1] = y[i] + dx * psi(f, a, b, x[i], x[i+1], y[i], dx)
    return x, y

def true_solution(x, a, b, c):
    return (c + a / (b * b)) * np.exp(b * x) - a / b * x - a / (b * b)

# 计算误差
def compute_error(y_true, y_approx):
    return np.sqrt(np.sum((y_true - y_approx)**2))

# 计算收敛阶
def compute_convergence_order(dx, error):
    order = np.polyfit(np.log(dx), np.log(error), 1)[0]
    return order

class Net(nn.Module):
    def __init__(
        self,
        layer_sizes,
        out_act = False,
        use_bias = True,
        act_layer = nn.ReLU,
    ):
        super().__init__()

        num_layers = len(layer_sizes) - 1
        assert num_layers >= 1, "num_layers must be greater than or equal to 1"

        self.layers = nn.ModuleList()
        for i in range(num_layers - 1):
            self.layers.append(
                nn.Linear(layer_sizes[i], layer_sizes[i + 1], bias=use_bias)
            )
            self.layers.append(act_layer())

        self.layers.append(
            nn.Linear(layer_sizes[-2], layer_sizes[-1], bias=use_bias)
        )
        if out_act:
            self.layers.append(act_layer())

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x.squeeze()

DHM_net = Net([4,80,80,80,80,80,80,80,80,1])
if os.path.exists("DHM.pth"):
    DHM_net.load_state_dict(torch.load("DHM.pth"))
    print("网络参数载入成功")
if torch.cuda.is_available():
    DHM_net = DHM_net.cuda()

# 初始化优化器
learning_rate = 5e-3
optimizer = torch.optim.Adam(DHM_net.parameters(), lr=learning_rate)

# 初始化损失函数
loss = nn.MSELoss()
loss = loss.cuda()

class TrainDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.inputs = data["inputs"]
        self.res = data["res"]

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.res[idx]

def deep_heun_method(f, a, b, c, dx, rangee, net):
    N = int((rangee[1] - rangee[0])/dx)
    x = np.zeros(N+1)
    y = np.zeros(N+1)
    x[0] = rangee[0]
    y[0] = c
    for i in range(N):
        x[i+1] = x[i] + dx
        input = torch.tensor([x[i], x[i + 1], y[i], psi(f, a, b, x[i], x[i+1], y[i], dx)]).float()
        y[i+1] = y[i] + dx * psi(f, a, b, x[i], x[i+1], y[i], dx) + dx**3 * net(input)
    return x, y

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_data["inputs"].shape

(2475000, 4)

In [17]:
# 生成数据
num_params = 500       # 生成参数数量
a = np.random.uniform(1, 5, num_params).astype(np.float32)
b = np.random.uniform(1, 5, num_params).astype(np.float32)
c = np.random.uniform(1, 5, num_params).astype(np.float32)

num_points = 100       # 每个参数对应离散点数量
train_x = []
train_y = []
for i in range(num_params):
    x = np.sort(np.random.uniform(rangee[0], rangee[1], num_points))
    train_x.append(x)

    true_y = true_solution(x, a[i], b[i], c[i])
    train_y.append(true_y)

x = np.array(train_x)
y = np.array(train_y)

In [18]:
def R(f, xi, xj, yi, yj, a, b):
    dx = abs(xj - xi)
    return (yj - yi - dx * psi(f, a, b, xi, xj, yi, dx)) / (dx**3)

res = []
inputs = []
for k, (_x, _y) in enumerate(zip(x, y)):
    for i in range(num_points):
        for j in range(i + 1, num_points):
            xi = _x[i]
            xj = _x[j]
            yi = _y[i]
            yj = _y[j]
            dx = abs(xj - xi)
            inputs.append(np.array([xi, xj, yi, psi(f, a[k], b[k], xi, xj, yi, dx)]))
            res.append(R(f, xi, xj, yi, yj, a[k], b[k]))
res = np.array(res).astype(np.float32)
inputs = np.array(inputs).astype(np.float32)

In [19]:
train_data = {"inputs": inputs, "res": res}
train_dataset = TrainDataset(train_data)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2048, shuffle=True)


In [38]:
np.sum(train_data["res"]>20000)

4